![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Read csv file 
rental_info_df = pd.read_csv('rental_info.csv')

# Check the dataset infor
print(rental_info_df.info())

# Convert the rental_date and retuen_date to datetime format
rental_info_df['rental_date'] = pd.to_datetime(rental_info_df['rental_date'])
rental_info_df['return_date'] = pd.to_datetime(rental_info_df['return_date'])

# Compute rental_length_days
rental_info_df['rental_length_days'] = (rental_info_df['return_date'] - rental_info_df['rental_date']).dt.days

# Create dummies variables from special features
rental_info_df['deleted_scenes'] = np.where(rental_info_df['special_features'].str.contains('Deleted Scenes'), 1, 0)
rental_info_df['behind_the_scenes'] = np.where(rental_info_df['special_features'].str.contains('Behind the Scenes'), 1, 0)

# Drop data leakage columns and irrelevant ones
drop_cols = ["rental_date", "return_date", "special_features", "rental_length_days"]
X = rental_info_df.drop(columns = drop_cols)
y = rental_info_df['rental_length_days']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Try different models
models = [
    ("LinearRegression", LinearRegression()),
    ("RandomForest", RandomForestRegressor(random_state=9)),
    ("GradientBoosting", GradientBoostingRegressor(random_state=9))
]

best_mse = float("inf")
best_model = None

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    if mse < best_mse:
        best_mse = mse
        best_model = model

# Print best model and MSE score
print(f"Best model: {type(best_model).__name__}")
print(f"Test MSE: {best_mse:.4f}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB
None
Best model: RandomForestR